In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import numpy as np
import bartz
from pyinstrument import Profiler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from bart_playground import *
import arviz as az

INFO:arviz.preview:arviz_base not installed
INFO:arviz.preview:arviz_stats not installed
INFO:arviz.preview:arviz_plots not installed


In [3]:
proposal_probs = {"multi_grow": 0.25, "multi_prune": 0.25, "multi_change": 0.4, "multi_swap": 0.1}
#proposal_probs = {"multi_grow": 0.5, "multi_prune": 0.5}
generator = DataGenerator(n_samples=160, n_features=2, noise=0.1, random_seed=42)
X, y = generator.generate(scenario="piecewise_flat")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [4]:
n_chains = 4
chains = []
rmse_chains = []
proposal_probs = {"multi_grow": 0.25, "multi_prune": 0.25, "multi_change": 0.4, "multi_swap": 0.1}
for i in range(n_chains):
    bart = MultiBART(ndpost=200, nskip=100, n_trees=100, proposal_probs=proposal_probs, multi_tries=10, random_state=i)
    bart.fit(X_train, y_train)
    sigmas = [trace.global_params['eps_sigma2'][0] for trace in bart.sampler.trace]
    preds = bart.posterior_f(X_test, backtransform=True)  # shape: (n_test, ndpost)
    rmses = [mean_squared_error(y_test, preds[:, k], squared=False) for k in range(preds.shape[1])]
    chains.append(sigmas)
    rmse_chains.append(rmses)

# chains: shape (n_chains, n_samples)
chains_array = np.array(chains)
idata = az.from_dict(posterior={"eps_sigma2": chains_array})
rhat = az.rhat(idata, var_names=["eps_sigma2"])
print("Gelman-Rubin R̂ (eps_sigma2):", rhat)

chains_array = np.array(rmse_chains)  # shape: (n_chains, n_samples)
idata = az.from_dict(posterior={"test_rmse": chains_array})
rhat = az.rhat(idata, var_names=["test_rmse"])
print("Gelman-Rubin R̂ (test_rmse):", rhat)

Iterations: 100%|██████████| 300/300 [00:09<00:00, 32.72it/s]


Gelman-Rubin R̂ (eps_sigma2): <xarray.Dataset> Size: 8B
Dimensions:     ()
Data variables:
    eps_sigma2  float64 8B 1.036
Gelman-Rubin R̂ (test_rmse): <xarray.Dataset> Size: 8B
Dimensions:    ()
Data variables:
    test_rmse  float64 8B 1.019


In [5]:
import pandas as pd

# Collect move counts
selected = bart.sampler.move_selected_counts
success = bart.sampler.move_success_counts
accepted = bart.sampler.move_accepted_counts

# Combine into a DataFrame for easy viewing
df = pd.DataFrame({
    "selected": pd.Series(selected),
    "success": pd.Series(success),
    "accepted": pd.Series(accepted)
})

# Add success and acceptance rates
df["success_rate"] = df["success"] / df["selected"]
df["mh_ratio"] = df["accepted"] / df["success"]
df["accept_rate"] = df["accepted"] / df["selected"]

print(df)

              selected  success  accepted  success_rate  mh_ratio  accept_rate
multi_grow        7612     7612      3522      1.000000  0.462690     0.462690
multi_prune       7520     7431      3170      0.988165  0.426591     0.421543
multi_change     11884    11728      9622      0.986873  0.820430     0.809660
multi_swap        2984     1992      1438      0.667560  0.721888     0.481903


In [6]:
rf = RandomForestRegressor(random_state=42)
lr = LinearRegression()
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)

btz = bartz.BART.gbart(np.transpose(X_train), y_train, ntree=100, ndpost=200, nskip=100)
btpred_all = btz.predict(np.transpose(X_test))
btpred = np.mean(np.array(btpred_all), axis=0)

INFO:2025-09-15 02:27:19,380:jax._src.xla_bridge:822: Unable to initialize backend 'tpu': UNIMPLEMENTED: LoadPjrtPlugin is not implemented on windows yet.
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': UNIMPLEMENTED: LoadPjrtPlugin is not implemented on windows yet.


....................................................................................................
It 100/300 grow P=60% A=40%, prune P=40% A=38%, fill=6% (burnin)
....................................................................................................
It 200/300 grow P=60% A=35%, prune P=40% A=38%, fill=6%
....................................................................................................
It 300/300 grow P=55% A=25%, prune P=45% A=29%, fill=6%


In [7]:
models = {"bart" : bart, 
          "rf" : rf, 
          "lr" : lr,
          "btz" : btz}
results = {}
for model_name, model in models.items():
    if model_name == "btz":
        results[model_name] = mean_squared_error(y_test, btpred)
    else:
        results[model_name] = mean_squared_error(y_test, model.predict(X_test))
results

{'bart': 0.023298654913038185,
 'rf': 0.022139023845392215,
 'lr': 0.048045521328019404,
 'btz': 0.02515917299472511}

## Default BART

In [8]:
# bart = DefaultBART(ndpost=50, nskip=0, n_trees=100, proposal_probs=proposal_probs)
# bart.fit(X_train, y_train)

In [9]:
n_chains = 4
chains = []
rmse_chains = []
proposal_probs = {"grow": 0.25, "prune": 0.25, "change": 0.4, "swap": 0.1}
for i in range(n_chains):
    bart = DefaultBART(ndpost=200, nskip=100, n_trees=100, proposal_probs=proposal_probs, random_state=i)
    bart.fit(X_train, y_train)
    sigmas = [trace.global_params['eps_sigma2'][0] for trace in bart.sampler.trace]
    preds = bart.posterior_f(X_test, backtransform=True)  # shape: (n_test, ndpost)
    rmses = [mean_squared_error(y_test, preds[:, k], squared=False) for k in range(preds.shape[1])]
    chains.append(sigmas)
    rmse_chains.append(rmses)

# chains: shape (n_chains, n_samples)
chains_array = np.array(chains)
idata = az.from_dict(posterior={"eps_sigma2": chains_array})
rhat = az.rhat(idata, var_names=["eps_sigma2"])
print("Gelman-Rubin R̂ (eps_sigma2):", rhat)

chains_array = np.array(rmse_chains)  # shape: (n_chains, n_samples)
idata = az.from_dict(posterior={"test_rmse": chains_array})
rhat = az.rhat(idata, var_names=["test_rmse"])
print("Gelman-Rubin R̂ (test_rmse):", rhat)

Iterations: 100%|██████████| 300/300 [00:01<00:00, 181.71it/s]


Gelman-Rubin R̂ (eps_sigma2): <xarray.Dataset> Size: 8B
Dimensions:     ()
Data variables:
    eps_sigma2  float64 8B 1.033
Gelman-Rubin R̂ (test_rmse): <xarray.Dataset> Size: 8B
Dimensions:    ()
Data variables:
    test_rmse  float64 8B 1.041


In [10]:
import pandas as pd

# Collect move counts
selected = bart.sampler.move_selected_counts
success = bart.sampler.move_success_counts
accepted = bart.sampler.move_accepted_counts

# Combine into a DataFrame for easy viewing
df = pd.DataFrame({
    "selected": pd.Series(selected),
    "success": pd.Series(success),
    "accepted": pd.Series(accepted)
})

# Add success and acceptance rates
df["success_rate"] = df["success"] / df["selected"]
df["mh_ratio"] = df["accepted"] / df["success"]
df["accept_rate"] = df["accepted"] / df["selected"]

print(df)

        selected  success  accepted  success_rate  mh_ratio  accept_rate
grow        7446     7446      3015      1.000000  0.404915     0.404915
prune       7425     6975      2861      0.939394  0.410179     0.385320
change     12088    11336      6797      0.937790  0.599594     0.562293
swap        3041      872       553      0.286748  0.634174     0.181848


In [11]:
models = {"bart" : bart, 
          "rf" : rf, 
          "lr" : lr,
          "btz" : btz}
results = {}
for model_name, model in models.items():
    if model_name == "btz":
        results[model_name] = mean_squared_error(y_test, btpred)
    else:
        results[model_name] = mean_squared_error(y_test, model.predict(X_test))
results

{'bart': 0.023852558410078183,
 'rf': 0.022139023845392215,
 'lr': 0.048045521328019404,
 'btz': 0.02515917299472511}